In [1]:
from dotenv import load_dotenv
import os
import json

from openai import OpenAI


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
!pip install minsearch

import minsearch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
# get the FAQ  documents
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

In [7]:
index.fit(documents)

In [8]:
open_ai_client = OpenAI()

In [9]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5 }
    
    results=index.search(
        query=query,
        # filter_dict={'course': 'data-engineering-zoomcamp'},
        filter_dict={'course': 'llm-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [58]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    context_template = """
    Q: {question}
    A: {text}
    """.strip()
    
    for response in search_results:
        context = context + context_template.format(question=response['question'], text=response['text']) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [11]:
def llm(prompt):
    response = open_ai_client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [12]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [13]:
query = "how do I run kafka"
answer = rag(query)
answer

'To assist you with running Kafka, I would typically need specific details provided in the context, such as the environment you are working in (Windows, MacOS, Linux) and whether Kafka is already installed. However, since no context detailing those specifics is provided, here are general steps for running Apache Kafka:\n\n1. **Install Apache Kafka**: Ensure you have Java installed, as Kafka requires Java to run. Download Apache Kafka from the official website and extract it to your desired location.\n\n2. **Start Zookeeper**: Kafka uses Zookeeper to manage distributed brokers. Navigate to the extracted Kafka directory and in a terminal, run:\n   ```\n   bin/zookeeper-server-start.sh config/zookeeper.properties\n   ```\n   On Windows, use `zookeeper-server-start.bat` instead of `zookeeper-server-start.sh`.\n\n3. **Start Kafka Server**: Open another terminal window, navigate to the Kafka directory, and start the Kafka broker by running:\n   ```\n   bin/kafka-server-start.sh config/server

In [14]:
from elasticsearch import Elasticsearch

In [15]:
es_client = Elasticsearch("http://localhost:9200")

In [16]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [21]:
from tqdm.auto import tqdm

for document in documents:
    es_client.index(index=index_name, document=document)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [45]:
def elastic_search(query):
    
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [46]:
query = "How do copy a file to a Docker container?"

In [115]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [54]:
elastic_search_response = elastic_search(query)

context = ""
context_template = """
Q: {question}
A: {text}
""".strip()

for response in elastic_search_response:
    context = context + context_template.format(question=response['question'], text=response['text']) + "\n\n"

In [60]:
prompt = build_prompt(query, elastic_search_response)

In [61]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How do copy a file to a Docker container?

    CONTEXT:
    Q: How do I debug a docker container?
    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
    A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory conta

In [62]:
len(prompt)

1474

In [63]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 35.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [67]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [68]:
num_tokens = len(encoding.encode(prompt))
num_tokens

327

In [71]:
#Bonus: generating the answer (ungraded)
answer = rag(query)
print(answer)

To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is:

```bash
docker cp <source_path> <container_name>:<destination_path>
```

Replace `<source_path>` with the path of the file on your host machine, `<container_name>` with the name or ID of the Docker container, and `<destination_path>` with the path inside the container where you want to copy the file.


In [74]:
#tokens in llm answer

num_tokens_anwer = len(encoding.encode(answer))
num_tokens_anwer

92

In [72]:
#Bonus: calculating the costs (ungraded)
#$5.00 / 1M input tokens
#$20.00 / 1M output tokens

#Suppose that on average per request we send 150 tokens and receive back 250 tokens.
#How much will it cost to run 1000 requests?

def total_cost(requests, avg_input_tokens, avg_output_tokens, price_per_million_input_tokens, price_per_million_output_tokens):
    return requests * (price_per_million_input_tokens*avg_input_tokens/1e6 + price_per_million_output_tokens*avg_output_tokens/1e6)

estimated_cost = total_cost(1000, 150, 250, 5, 20)
estimated_cost

5.75

In [73]:
# estimated cost from actual question and response:

# Q: -> prompt: 327 tokens
# Answer: answer: 91 tokens

In [76]:
estimated_cost = total_cost(1, 327, 91, 5, 20)
estimated_cost

0.0034549999999999997